<a href="https://colab.research.google.com/github/ProStaffRF/LA-Bench2025_test/blob/main/baseline_3separte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# Cell 1: 環境設定とセットアップ
"""
LA-Bench 2025: 実験手順生成タスク
Baseline Implementation for Google Colaboratory
GitHub: https://github.com/lasa-or-jp/la-bench.git
"""

#@title 1. 環境セットアップ { display-mode: "form" }
#@markdown このセルを実行して必要なライブラリをインストールし、リポジトリをクローンします。


import os
from pathlib import Path

# Colabかどうかの確認
try:
    import google.colab
    IN_COLAB = True
    print("✅ Google Colaboratory環境を検出しました")
    # 必要なライブラリのインストール
    print("\n📦 必要なライブラリをインストール中...")
    !pip install -q openai pyyaml tqdm pandas

    print("✅ ライブラリのインストール完了")

    # GitHubリポジトリのクローン
    REPO_URL = "https://github.com/lasa-or-jp/la-bench.git"
    REPO_NAME = "la-bench"

    if not os.path.exists(REPO_NAME):
        print(f"\n📥 リポジトリをクローン中: {REPO_URL}")
        !git clone -q {REPO_URL}
        print(f"✅ リポジトリのクローン完了: {REPO_NAME}/")
    else:
        print(f"\n📂 リポジトリは既に存在します: {REPO_NAME}/")
        print("📥 最新版に更新中...")
        !cd {REPO_NAME} && git pull -q
        print("✅ 更新完了")

    # 作業ディレクトリの設定
    WORK_DIR = Path(REPO_NAME)
    os.chdir(WORK_DIR)
    print(f"\n📍 作業ディレクトリ: {os.getcwd()}")

    # ディレクトリ構造の確認
    print("\n📊 プロジェクト構造:")
    !ls -la
except ImportError:
    IN_COLAB = False
    print("⚠️ ローカル環境で実行中です")
    if Path.cwd().name == "notebooks":
        os.chdir(Path.cwd().parent)

✅ Google Colaboratory環境を検出しました

📦 必要なライブラリをインストール中...
✅ ライブラリのインストール完了

📥 リポジトリをクローン中: https://github.com/lasa-or-jp/la-bench.git
✅ リポジトリのクローン完了: la-bench/

📍 作業ディレクトリ: /content/la-bench/la-bench/la-bench

📊 プロジェクト構造:
total 420
drwxr-xr-x  9 root root   4096 Nov  9 06:12 .
drwxr-xr-x 11 root root   4096 Nov  9 06:12 ..
drwxr-xr-x  2 root root   4096 Nov  9 06:12 announcements
-rw-r--r--  1 root root   4741 Nov  9 06:12 CLAUDE.md
drwxr-xr-x  4 root root   4096 Nov  9 06:12 code
-rw-r--r--  1 root root   2658 Nov  9 06:12 CONTRIBUTING.md
drwxr-xr-x  4 root root   4096 Nov  9 06:12 data
drwxr-xr-x  3 root root   4096 Nov  9 06:12 docs
drwxr-xr-x  8 root root   4096 Nov  9 06:12 .git
drwxr-xr-x  3 root root   4096 Nov  9 06:12 .github
-rw-r--r--  1 root root    979 Nov  9 06:12 .gitignore
-rw-r--r--  1 root root   1101 Nov  9 06:12 LICENSE
drwxr-xr-x  2 root root   4096 Nov  9 06:12 notebooks
-rw-r--r--  1 root root    569 Nov  9 06:12 pyproject.toml
-rw-r--r--  1 root root  12425 Nov  9 0

In [24]:
# Cell 2: OpenAI APIキーの設定
#@title 2. OpenAI API Key設定 { display-mode: "form" }
#@markdown OpenAI APIキーを入力してください。キーは安全に管理されます。


# APIキーの取得方法を選択
use_secrets = False  #@param {type:"boolean"}
#@markdown ☝️ Google Colab Secretsを使用する場合はチェック

if IN_COLAB:
    import getpass
    from google.colab import userdata
    if use_secrets:
        try:
            # Colab Secretsから取得
            API_KEY = userdata.get('OPENAI_API_KEY')
            print("✅ APIキーをSecretsから取得しました")
        except Exception as e:
            print("⚠️ Secretsからの取得に失敗しました")
            print("左側のパネルの🔑アイコンから'OPENAI_API_KEY'を設定してください")
            API_KEY = None
    else:
        # 直接入力
        api_key_input = getpass.getpass("🔑 OpenAI API Keyを入力: ")
        if api_key_input:
            API_KEY = api_key_input
            os.environ['OPENAI_API_KEY'] = API_KEY
            print("✅ APIキーが設定されました")
        else:
            API_KEY = None
            print("⚠️ APIキーが設定されていません（ヒューリスティック手法のみ使用）")
else:
    # ローカル環境の場合
    API_KEY = os.getenv("OPENAI_API_KEY")
    if not API_KEY:
        API_KEY = input("OpenAI API Key: ")

# APIキーの検証
if API_KEY:
    print(f"🔑 APIキー: {'*' * 20}{API_KEY[-4:]}")
else:
    print("⚠️ GPT機能は使用できません")

🔑 OpenAI API Keyを入力: ··········
✅ APIキーが設定されました
🔑 APIキー: ********************Ct8A


In [25]:
# Cell 3: ライブラリのインポートと設定
#@title 3. ライブラリのインポート { display-mode: "form" }

import json
import yaml
import time
from datetime import datetime
from typing import Dict, List, Optional, Tuple, Set, Any
from pathlib import Path
from copy import deepcopy
import warnings
warnings.filterwarnings('ignore')

# データ処理
import pandas as pd
from dataclasses import dataclass, field

# OpenAI API
try:
    from openai import OpenAI
    OPENAI_AVAILABLE = True
except ImportError:
    OPENAI_AVAILABLE = False
    print("⚠️ OpenAIライブラリが利用できません")

# プログレスバー (Colab対応)
from tqdm.auto import tqdm

# ログ設定
import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%H:%M:%S'
)
logger = logging.getLogger(__name__)

print("="*60)
print("LA-Bench 2025 Baseline Implementation")
print(f"実行環境: {'Google Colab' if IN_COLAB else 'Local'}")
print(f"実行時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"OpenAI利用可能: {OPENAI_AVAILABLE and API_KEY is not None}")
print("="*60)

LA-Bench 2025 Baseline Implementation
実行環境: Google Colab
実行時刻: 2025-11-09 06:12:40
OpenAI利用可能: True


In [26]:
# Cell 4: データ構造
#@title 4. データ構造の定義 { display-mode: "form" }

@dataclass
class Step:
    id: int
    text: str

@dataclass
class ReferenceEntry:
    id: int
    text: str

@dataclass
class ExampleInput:
    instruction: str
    mandatory_objects: Set[str] = field(default_factory=set)
    source_protocol_steps: List[Step] = field(default_factory=list)
    expected_final_states: Set[str] = field(default_factory=set)
    references: List[ReferenceEntry] = field(default_factory=list)

@dataclass
class ExampleOutput:
    procedure_steps: List[Step] = field(default_factory=list)

@dataclass
class Measurement:
    specific_criteria: Dict[str, int] = field(default_factory=dict)

@dataclass
class ExampleSample:
    id: str
    input: ExampleInput
    output: ExampleOutput
    measurement: Optional[Measurement] = None

def _to_set(x):
    return set(x) if isinstance(x, (list, set, tuple)) else set()

def _to_list(x):
    return list(x) if isinstance(x, (list, set, tuple)) else (x if isinstance(x, list) else [])

def _to_steps(x) -> List[Step]:
    steps: List[Step] = []
    arr = _to_list(x)
    if not arr:
        return steps
    if isinstance(arr[0], dict):
        for it in arr:
            try:
                sid = int(it.get("id", len(steps) + 1))
            except Exception:
                sid = len(steps) + 1
            steps.append(Step(id=sid, text=str(it.get("text", "")).strip()))
    else:
        for idx, s in enumerate(arr, start=1):
            steps.append(Step(id=idx, text=str(s).strip()))
    return steps

def _to_references(x) -> List[ReferenceEntry]:
    refs: List[ReferenceEntry] = []
    arr = _to_list(x)
    if not arr:
        return refs
    if isinstance(arr[0], dict):
        for it in arr:
            try:
                rid = int(it.get("id", len(refs) + 1))
            except Exception:
                rid = len(refs) + 1
            refs.append(ReferenceEntry(id=rid, text=str(it.get("text", "")).strip()))
    else:
        for idx, ref in enumerate(arr, start=1):
            refs.append(ReferenceEntry(id=idx, text=str(ref).strip()))
    return refs

def parse_sample(obj: Dict[str, Any]) -> ExampleSample:
    sid = obj.get("id") or obj.get("sample_id") or "unknown"
    i = obj.get("input", {})
    o = obj.get("output", {})
    m = obj.get("measurement", {})

    # Measurement.specific_criteria を dict に正規化（list形式も許容）
    sc_raw = m.get("specific_criteria", {})
    sc: Dict[str, int] = {}
    if isinstance(sc_raw, dict):
        for k, v in sc_raw.items():
            try:
                sc[str(k)] = int(v)
            except Exception:
                pass
    elif isinstance(sc_raw, list):
        for it in sc_raw:
            try:
                k = it.get("item")
                v = int(it.get("score", 0))
                if k:
                    sc[str(k)] = v
            except Exception:
                pass

    sample = ExampleSample(
        id=str(sid),
        input=ExampleInput(
            instruction=str(i.get("instruction", "")).strip(),
            mandatory_objects=_to_set(i.get("mandatory_objects", [])),
            source_protocol_steps=_to_steps(i.get("source_protocol_steps", [])),
            expected_final_states=_to_set(i.get("expected_final_states", [])),
            references=_to_references(i.get("references", [])),
        ),
        output=ExampleOutput(
            procedure_steps=_to_steps(o.get("procedure_steps", []))
        ),
        measurement=Measurement(specific_criteria=sc) if sc else None
    )
    return sample

def load_example_jsonl(path: str):
    samples = []
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"JSONL not found: {p}")
    for line in p.read_text(encoding="utf-8").splitlines():
        line = line.strip()
        if not line:
            continue
        try:
            obj = json.loads(line)
        except Exception as e:
            print(f"⚠️ JSONL parse error: {e}")
            continue
        samples.append(parse_sample(obj))
    return samples

In [27]:
# Cell 5: JSONLローダーの利用
#@title 5. JSONLファイルを読み込む { display-mode: "form" }
#@markdown exampleを使うとき：`data/example/example.jsonl`
#@markdown public_testを使うとき：`data/public_test/public_test.jsonl`

jsonl_path = 'data/public_test/public_test.jsonl'  #@param {type:'string'}

try:
    samples = load_example_jsonl(jsonl_path)
    print(f'✅ Loaded {len(samples)} samples from {jsonl_path}')
except Exception as e:
    print(f'❌ Load error: {e}')


✅ Loaded 10 samples from data/public_test/public_test.jsonl


In [32]:
# Cell 6: 実験手順の生成（OpenAI, Pydantic構造化）
#@title 6. LLMで Input から Output（procedure_steps）を生成 { display-mode: "form" }

# --- 必要なライブラリのインポート ---
from pydantic import BaseModel, Field
from typing import List, Dict, Any, Set, Optional # 型ヒントのため
import json # JSONL保存に必要
from pathlib import Path # Pathオブジェクトに必要
import time # タイムスタンプに必要
from openai import OpenAI # OpenAIクライアントのインポート (Cell 5でインポート済みと想定)

# --- 🧪 モデル設定の最適化 ---
MODEL_NAME = "gpt-4o-2024-08-06" #@param ["gpt-4.1-mini-2025-04-14", "gpt-4o-2024-08-06", "gpt-5-2025-08-07", "gpt-5-mini-2025-08-07", "gpt-5-nano-2025-08-07"]
TEMPERATURE = 0.1 # @param {type:"number"} <- 安定性のため0.1に設定

#@markdown `build_messages`関数において、LLMの入力を設計しています。

# --- Pydanticデータ構造（変更なし）---
class StepModel(BaseModel):
    id: int = Field(ge=1, description="ステップ番号")
    text: str = Field(description="実験手順の詳細な説明")

class GeneratedOutput(BaseModel):
    procedure_steps: List[StepModel] = Field(
        description="実験手順のリスト",
        min_items=1,
        max_items=50
    )

# --- 💡 分野別厳格化仕様書セクションの定義 (Specific Score強化版) ---

MOLBIO_SPEC = """
## 分子生物学プロトコル厳格化仕様書 (Specific Score強化)
I. 計測パラメータと曖昧表現の定義:
- ルール 1.2.1: 「しばらく」は標準デフォルト時間として 5.0 min を適用する。
- ルール 1.3.2: 「氷上」は 4.0℃（許容範囲 0.0〜4.0℃）の冷却ブロックを強制する。
- **[新規S-1.4: 希釈計画]** プライマーや高濃度核酸ストック（10 μM以上）を反応に用いる場合、**必ず中間ストック濃度への希釈（例: 100 μM → 10 μM）の手順と、その具体的な計算**を明記すること。
- **[新規S-1.5: 測定条件]** UV/Vis測定（260 nmなど）を行うステップでは、使用する**標準セル/プレートタイプ、読み取りモード（例: ボトムリード）**、および**測定前の液面均一化手順（例: 5秒間の低速混合）**を詳細に記述すること。

II. 操作の粒度と順序の厳格化:
- ルール 2.1.1: 「添加・混合」は、Pipette, Mix, Spin Down (500rcf) の3ステップシーケンスに強制的に分解される。
- **[新規S-2.2: マスターミックス構成]** マスターミックス調製の手順では、**最終的な各構成要素の濃度**と**計算に使用したストック濃度**を手順中にカッコ書きで記載することを義務付ける。

III. 安全・環境に関する暗黙知:
- ルール 3.3.2: 核酸を扱うワークフロー実行前後に、デッキ表面のUV除染サイクルを強制的に挿入する。
"""

BIOCHEM_SPEC = """
## 生化学・タンパク質精製プロトコル厳格化仕様書 (Specific Score強化)
I. 標準的な計測パラメータと単位の厳格化:
- ルール I.B: 遠心力は常にrcf単位（g）で記述されなければならない。
- ルール II.A: 「一晩 (Overnight)」は 16 hours を強制デフォルトとし、自動的に $4^{\circ} \text{C}$ 低温保持を連結する。
- **[新規S-1.4: 固形試薬の溶解]** PEG 4000や高分子量化合物などの固形試薬の溶解は、単なる「添加」ではなく、**溶解を保証するため最低15分間の穏やかな攪拌**（例: 500 rpm ロッキング）を強制挿入する。

II. 操作の粒度と順序の自動化規則:
- ルール III.A: 「試薬を添加し、混合する」は、Aspirate、Dispense、Mix_Actionの3つの独立したLAステップに厳格に分解されることが強制される。
- **[新規S-2.2: 測定前のQC]** 光学的測定（BCA、吸光度など）を行う直前には、**気泡の有無を確認（Visual Check）**し、その後の測定ステップに「気泡なしの記録 (QC Log)」を強制的に付随させる。

III. 安全性、環境制約、および機器設定:
- ルール IV.A: 全ての酵素、抗体、精製タンパク質溶液は、LAデッキ上で常に**4°C以下のクーリングブロック**での保管を強制する。
- **[新規S-3.2: PPE]** 危険試薬の取り扱い時には、手順中に**「手袋、保護眼鏡の着用を確認」**という安全確認ステップを明記する。
"""

BIOLOGY_SPEC = """
## 生物実験分野の規範的ルールセット (Specific Score強化)
I. 標準的な計測パラメータの厳格な標準化ルール:
- ルール 1.1.1: 「しばらく」は標準デフォルト時間として 300秒（5分）を適用する。
- ルール 1.2.1: 細胞ペレット化の標準デフォルト条件は 150 x g で 5 minutes を強制する。

II. 操作の粒度とシーケンスの自動化規則:
- ルール 2.1.2: 「細胞の状態を確認する」は、自動イメージングによるコンフルエンス $\mathbf{\ge 70\%}$ の確認などの定量的測定ステップに強制的に変換される。
- **[新規S-2.4: イメージングパラメータ]** 自動イメージング（蛍光、明視野）を行うステップでは、使用する**フィルターセット（例: FITC/TRITC）**と**露光時間（例: 50 ms）**を具体的な数値で記述することを義務付ける。

III. 安全および環境に関する暗黙知のプロトコル組み込み:
- ルール 3.1.1: 細胞培養関連試薬は、添加直前の温度を 37.0 ± 0.5 °C に維持することを強制する。
- **[新規S-3.2: 試料リスク]** 感染リスクのある試料（例: 特定の細胞株、ウイルス）を扱うプロトコルの場合、使用する**試験機（LAロボット）のリスクレベル**を手順書の冒頭に明記する。
"""

ORGANIC_SPEC = """
## 有機化学ラボオートメーションシステムのためのプロトコル生成ルール (Specific Score強化)
I. 標準化された計測パラメータと曖昧表現の厳格な規定:
- ルール 1.2.1: 「終夜」は $T_{\text{overnight}} = 14$ 時間を標準とする。LAはインライン分析の結果 (転化率 $X > 95\%$) を優先する動的調整ロジックを強制する。
- ルール 1.2.3: 「ゆっくり滴下」は標準滴下速度 $\mathbf{0.5 \sim 1.0 \, \text{mL/min}}$ の範囲を標準とする。

II. 操作の粒度、順序制御、および標準的な前処理プロトコル:
- ルール 2.1.1: 「添加・混合」は、分注、チップ洗浄/排出、混合の最低3つの原子ステップに分解することを義務付ける。

III. 安全・環境に関する暗黙知と実行制約:
- ルール 3.1.3: 水/空気感受性試薬は、分注ステップの前後 $\mathbf{5}$ 秒間、チップ先端に不活性ガス（$\text{N}_{2}$）の保護シールドを吹き付ける**「ガスシールド分注」**を強制的に挿入する。
- **[新規S-3.4: 廃棄物記録]** 廃棄物処理ステップでは、廃棄物容器の「反応履歴クラス」と**排出量（gまたはmL）**をログに記録することを義務付ける。
"""

INORGANIC_SPEC = """
## 無機化学実験分野における暗黙知の抽出と数値化仕様書 (Specific Score強化)
I. 標準的な計測パラメータと曖昧表現の数値的定義:
- ルール 1.1.3: 濃度の標準化: $\text{ppm}$ は分子量データに基づき $\text{M}$ への**動的換算**を内部で行うことを義務付ける。

II. 操作の原子分解とプロトコル順序の厳格化:
- ルール 2.3: 液-液抽出は、吸引前に**$120 \text{ s}$ 以上の強制静置ステップ**を組み込む。

III. 安全・環境に関する暗黙知のプロトコル組み込み:
- ルール 3.1.2: 発熱反応時の強制氷浴: 濃硫酸や高濃度水酸化物溶液の混合は、**$0^\circ \text{C}$ から $4^\circ \text{C}$ の氷浴**中で行うことを強制する。
"""

# --- 💡 全仕様書を格納するマスター辞書 ---
ALL_FIELD_SPECS = {
    "分子生物学": MOLBIO_SPEC,
    "生化学・タンパク質精製": BIOCHEM_SPEC,
    "生物実験": BIOLOGY_SPEC,
    "有機化学": ORGANIC_SPEC,
    "無機化学": INORGANIC_SPEC,
}

# --- 💡 分野特定関数の定義（変更なし） ---
def classify_experiment_field(instruction: str) -> str:
    """実験指示に含まれるキーワードに基づき、適用すべき専門分野を特定する"""
    instruction_lower = instruction.lower()

    # 1. 生物実験 (細胞、培養) を最優先にチェック
    if any(keyword in instruction_lower for keyword in ["細胞", "培地", "培養", "継代", "トリプシン", "コンフルエンス", "ウェル", "インキュベート"]):
        return "生物実験"

    # 2. 有機化学をチェック (合成、溶媒、不活性雰囲気など)
    elif any(keyword in instruction_lower for keyword in ["合成", "有機", "還流", "触媒", "溶媒", "加熱", "滴下", "不活性", "グローブボックス"]):
        return "有機化学"

    # 3. 無機化学をチェック (沈殿、金属、酸/塩基、モル量計算、極微量)
    elif any(keyword in instruction_lower for keyword in ["無機", "錯体", "沈殿", "金属", "比色", "ppm", "ppb", "酸", "塩基", "水酸化物", "ガス定量"]):
        return "無機化学"

    # 4. 生化学・タンパク質精製をチェック
    elif any(keyword in instruction_lower for keyword in ["タンパク質", "精製", "アッセイ", "bca", "クロマト", "透析", "変性", "カラム", "SDS-PAGE", "酵素"]):
        return "生化学・タンパク質精製"

    # 5. 分子生物学をチェック
    elif any(keyword in instruction_lower for keyword in ["dna", "rna", "pcr", "プライマー", "電気泳動", "核酸", "制限酵素"]):
        return "分子生物学"

    # 6. どの分野にもマッチしない場合のデフォルト分野
    return "生化学・タンパク質精製"

# --- 💡 build_messages 関数の修正（動的選択ロジック） ---
def build_messages(sample: ExampleSample) -> list[dict]:

    # --- 修正後の基本システムプロンプト (計画性と安全性の強制) ---
    base_sys_prompt = (
        "あなたは、化学、生物学、および材料科学を含む全分野の**ラボオートメーション (LA) プロトコル変換の最高専門家**です。"
        "あなたの唯一の任務は、提供された Input を読み込み、LAシステムが実行可能な**極めて厳格で詳細な実験手順**（procedure_steps）を返すことです。"
        "最重要制約: 出力は、Pydanticスキーマに厳格に準拠した**JSONオブジェクトのみ**であり、いかなる説明やコードブロックも許されません。"
        "制約: ステップ数は最大50、各ステップは10文以下、idは1から昇順でなければなりません。"

        # --- ★ Specific Score対策の全分野共通ルール ★ ---
        "\n\n### 全分野共通の実行前計画および品質管理 (Specific Score優先)"
        "A. **計画と計算の明示**: 複雑な試薬（マスターミックス、希釈液）調製を行うステップの直前には、**必要な最終容量と各試薬のストック濃度からの具体的な計算手順**を、**必ず**手順書に盛り込むこと。"
        "B. **サンプルハンドリングの分解**: 実験開始時、サンプルの初期容器からLAシステムの使用する新しい容器（マイクロチューブなど）への移動は、必ず**個別の手順**として記述すること。"
        "C. **PPEと安全性**: 全ての操作は、**適切なPPE（例: 手袋、保護眼鏡）の着用**から開始し、危険物（溶媒、酸）の操作時には**ドラフト（LEV）の確認**を手順に組み込むこと。"
    )

    # 1. 実験指示から分野を特定
    identified_field = classify_experiment_field(sample.input.instruction)

    # 2. 関連する厳格仕様書をマスター辞書から取得
    FIELD_SPECIFICATION = ALL_FIELD_SPECS.get(identified_field, "")

    # 3. 基本プロンプトと仕様書を結合
    if FIELD_SPECIFICATION:
        augmented_sys_prompt = base_sys_prompt + (
            f"\n\n### 専門分野プロトコル厳格化仕様書 - 実行義務: {identified_field} \n"
            "あなたは、この仕様書に記載されている**すべてのルールと具体的な数値定義を最優先で適用**し、曖昧な指示を仕様書に従って**強制的に数値パラメータに変換**する必要があります。\n"
            f"\n{FIELD_SPECIFICATION}" # 選択された仕様書全体をプロンプトに挿入
        )
    else:
        # 仕様書が見つからない場合は、修正された基本プロンプトのみを使用
        augmented_sys_prompt = base_sys_prompt

    # 4. ユーザープロンプト (usr) の作成（変更なし）
    user_lines = []
    user_lines.append(f"【実験指示】\n{sample.input.instruction}")

    if sample.input.mandatory_objects:
        user_lines.append("\n【使用する物品】")
        for it in sorted(sample.input.mandatory_objects):
            user_lines.append(f"- {it}")
    if sample.input.source_protocol_steps:
        user_lines.append("\n【元プロトコルの手順（参考）】")
        for st in sample.input.source_protocol_steps:
            user_lines.append(f"- {st.id}. {st.text}")
    if sample.input.expected_final_states:
        user_lines.append("\n【期待される最終状態】")
        for fs in sorted(sample.input.expected_final_states):
            user_lines.append(f"- {fs}")
    if sample.input.references:
        user_lines.append("\n【参考文献】")
        for ref in sample.input.references:
            user_lines.append(f"- [{ref.id}] {ref.text}")

    usr = "\n".join(user_lines)

    # 5. メッセージリストの作成
    return [
        {"role": "system", "content": augmented_sys_prompt},
        {"role": "user", "content": usr},
    ]

# --- API呼び出しロジックの復元 ---
def generate_outputs(samples: list[ExampleSample]) -> list[dict]:

    # ダミー処理を防ぐため、OpenAIクライアントの初期化を試行
    try:
        # API_KEYは外部で定義済みと想定
        client = OpenAI(api_key=API_KEY)
        print("📢 API呼び出しを開始します。実行には時間がかかる場合があります...")
    except NameError:
        print("❌ エラー: API_KEYまたはOpenAIクライアントが定義されていません。処理を中断します。")
        return []

    results: list[dict] = []

    for sm in samples:
        msgs = build_messages(sm)
        try:
            # 本番環境のロジック: LLMを呼び出し、Structured Outputでパース
            completion = client.chat.completions.parse(
                model=MODEL_NAME,
                messages=msgs,
                temperature=TEMPERATURE,
                response_format=GeneratedOutput,
            )
            parsed: GeneratedOutput = completion.choices[0].message.parsed  # type: ignore
            steps = [
                Step(id=s.id, text=s.text)
                for s in sorted(parsed.procedure_steps, key=lambda x: x.id)
            ][:50]
        except Exception as e:
            # API呼び出し失敗時のエラー処理
            print(f"❌ 生成失敗: {sm.id}: {e}")
            steps = []
        results.append({
            "id": sm.id,
            "procedure_steps": [{"id": s.id, "text": s.text} for s in steps],
        })

    if len(samples) > 0:
      print(f"✅ 生成完了: {len(results)} samples")
      print(f"例: {results[0]['id']} → {len(results[0]['procedure_steps'])} steps")

    return results

# 実行
# 'samples' 変数がCell 5以前でロードされていることを前提とします。
if 'samples' in globals():
    generated_results = generate_outputs(samples)
else:
    print("❌ エラー: 採点に必要な 'samples' 変数が見つかりません。Cell 5以前を実行してください。")
    generated_results = []

# 生成結果を JSONL で保存し、ダウンロードリンクを表示
ts = time.strftime('%Y%m%d_%H%M%S')
out_dir = Path('./outputs/runs')
out_dir.mkdir(parents=True, exist_ok=True)
jsonl_path = out_dir / f'generated_{ts}.jsonl'

if generated_results:
    # ファイル保存
    with jsonl_path.open('w', encoding='utf-8') as f:
        for rec in generated_results:
            obj = {"id": rec["id"], "output": {"procedure_steps": rec["procedure_steps"]}}
            line = json.dumps(obj, ensure_ascii=False, separators=(",", ":"))
            f.write(line + "\n")
    print(f"📄 Saved JSONL: {jsonl_path}")

    # ダウンロード（Colab/ローカル双方に対応）
    try:
        from google.colab import files as colab_files  # type: ignore

        # Colab環境: ダウンロードを強制実行
        print(f"\n--- ⬇️ ダウンロード実行中 ---")
        print(f"ファイル名: {jsonl_path.name}")
        colab_files.download(str(jsonl_path))
        print(f"✅ ダウンロード完了。お使いのPCのダウンロードフォルダを確認してください。")

    except Exception:
        # Colab環境外（ローカル環境など）: リンク表示
        from IPython.display import FileLink, display
        print(f"\n--- 📄 ローカルファイルリンク ---")
        display(FileLink(str(jsonl_path.resolve())))
        print("👆 上記のリンクをクリックしてファイルをダウンロードしてください。")

📢 API呼び出しを開始します。実行には時間がかかる場合があります...
✅ 生成完了: 10 samples
例: public_test_1 → 19 steps
📄 Saved JSONL: outputs/runs/generated_20251109_062620.jsonl

--- ⬇️ ダウンロード実行中 ---
ファイル名: generated_20251109_062620.jsonl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ ダウンロード完了。お使いのPCのダウンロードフォルダを確認してください。


In [33]:
# Cell 7: LLM-as-a-judge 評価（10点満点）
#@title 7. LLM で共通5点 + 個別5点を採点 { display-mode: "form" }

import time
import pandas as pd
from typing import List, Optional
from pydantic import BaseModel, Field
from pathlib import Path

try:
    from openai import OpenAI
except Exception as e:
    raise RuntimeError("OpenAI SDK v1 が見つかりません。`uv add openai` で追加してください。") from e

try:
    # Colabでのダウンロード機能を使用するためインポート
    from google.colab import files as colab_files
    from google.colab.output import eval_js
except ImportError:
    # Colab環境外の場合は無視
    pass

JUDGE_MODEL = "gpt-4o-2024-08-06" # 評価モデルを高性能モデルに設定
JUDGE_TEMPERATURE = 0.2

class JudgeOutput(BaseModel):
    general_score: float = Field(ge=0, le=5)
    specific_score: float = Field(ge=0, le=5)
    final_score: float = Field(ge=0, le=10)
    general_reason: str
    specific_matches: List[str] = []
    notes: Optional[str] = None

def build_judge_messages(sample: ExampleSample, steps: List[Step]) -> list[dict]:
    # 評価基準（共通5点 + 個別5点）
    system = (
        "あなたは生命科学実験の専門家であり、公平な採点者です。"
        "以下の基準に従って、与えられた Input と生成手順（Output）を評価し、"
        "general_score(0-5) と specific_score(0-5) と final_score(0-10) を出力してください。"
        "\n\n[共通採点基準 5点満点]\n"
        "加点(+1ずつ): 1) 実験指示のパラメータ反映, 2) 使用する物品の反映, 3) 元手順の論理反映, 4) 期待される最終状態の達成, 5) 適切な補完。\n"
        "減点: 不自然な日本語/ハルシネーション, 計算ミス, 手順矛盾。\n"
        "上限: 入力手順の丸写し等の過度の安全性が見られる場合、general_score は最大2点に制限。\n\n"
        "[個別採点基準 5点満点]\n"
        "与えられた specific_criteria の各 item が手順に含まれる/満たすなら、その score を加点（合計5点で上限）。"
    )

    parts = []
    parts.append(f"【実験指示】\n{sample.input.instruction}")
    if sample.input.mandatory_objects:
        parts.append("\n【使用する物品】")
        for it in sorted(sample.input.mandatory_objects):
            parts.append(f"- {it}")
    if sample.input.source_protocol_steps:
        parts.append("\n【元プロトコルの手順（参考）】")
        for st in sample.input.source_protocol_steps:
            parts.append(f"- {st.id}. {st.text}")
    if sample.input.expected_final_states:
        parts.append("\n【期待される最終状態】")
        for fs in sorted(sample.input.expected_final_states):
            parts.append(f"- {fs}")
    if sample.input.references:
        parts.append("\n【参考文献】")
        for ref in sample.input.references:
            parts.append(f"- [{ref.id}] {ref.text}")

    parts.append("\n【生成手順（Output）】")
    for s in steps:
        parts.append(f"- {s.id}. {s.text}")

    parts.append("\n【specific_criteria】")
    if sample.measurement and sample.measurement.specific_criteria:
        for item, sc in sample.measurement.specific_criteria.items():
            parts.append(f"- ({int(sc)}点) {item}")
    else:
        parts.append("- なし")

    user = "\n".join(parts)
    return [
        {"role": "system", "content": system},
        {"role": "user", "content": user},
    ]

def judge_with_llm(samples: List[ExampleSample], generated: list[dict]) -> pd.DataFrame:
    # API_KEYの存在を確認
    if 'API_KEY' not in globals() or not globals()['API_KEY']:
        print("❌ API_KEYが未設定です。採点処理をスキップします。")
        return pd.DataFrame() # 空のデータフレームを返す

    client = OpenAI(api_key=API_KEY)
    proc_map = {g['id']: [Step(id=it['id'], text=it['text']) for it in g['procedure_steps']] for g in generated}
    rows = []
    quota_exhausted = False

    print(f"📢 評価を開始します。{len(samples)}個のサンプルを採点します...")

    def _is_insufficient_quota(err: Exception) -> bool:
        s = str(err).lower()
        return 'insufficient_quota' in s or 'you exceeded your current quota' in s

    for sm in samples:
        if quota_exhausted:
            print(f"⏭️ スキップ採点: {sm.id}（クォータ不足）")
            rows.append({
                'id': sm.id,
                'general_score': 0.0,
                'specific_score': 0.0,
                'total_score': 0.0,
                'notes': 'skipped_due_to_quota',
            })
            continue

        # 生成手順が存在しない場合は採点スキップ
        steps = proc_map.get(sm.id, [])
        if not steps:
            print(f"⏭️ スキップ採点: {sm.id}（生成手順なし）")
            rows.append({
                'id': sm.id,
                'general_score': 0.0,
                'specific_score': 0.0,
                'total_score': 0.0,
                'notes': 'skipped_no_generated_steps',
            })
            continue

        msgs = build_judge_messages(sm, steps)
        try:
            completion = client.chat.completions.parse(
                model=JUDGE_MODEL,
                messages=msgs,
                temperature=JUDGE_TEMPERATURE,
                response_format=JudgeOutput,
            )
            parsed: JudgeOutput = completion.choices[0].message.parsed  # type: ignore
            rows.append({
                'id': sm.id,
                'general_score': parsed.general_score,
                'specific_score': parsed.specific_score,
                'total_score': parsed.final_score,
                'notes': parsed.notes or '',
            })
        except Exception as e:
            print(f"❌ 評価失敗: {sm.id}: {e}")
            if _is_insufficient_quota(e):
                print("⚠️ APIクォータ不足のため、以降の採点を中断します。プラン/課金設定をご確認ください。")
                quota_exhausted = True
            rows.append({
                'id': sm.id,
                'general_score': 0.0,
                'specific_score': 0.0,
                'total_score': 0.0,
                'notes': 'evaluation_failed',
            })
    return pd.DataFrame(rows)

# 実行
# 'samples'と'generated_results'がCell 6までに定義されていることを前提とする
if 'samples' in globals() and 'generated_results' in globals():
    df = judge_with_llm(samples, generated_results)

    if not df.empty:
        print(f"✅ LLM-as-a-judge: Scored {len(df)} samples (0-10)")
        try:
            # Colab/Jupyter環境でのDataFrame表示
            display(df[['id','general_score','specific_score','total_score']])
        except Exception:
            # その他環境での単純表示
            print(df[['id','general_score','specific_score','total_score']])

        # CSVファイルの保存
        # 'out_dir'と'ts'がCell 6から継続していることを前提とする
        csv_path = out_dir / f'eval_llm_{ts}.csv'
        df.to_csv(csv_path, index=False, encoding="utf_8_sig")
        print(f'📄 Saved: {csv_path}')

        # ダウンロード（Colab/ローカル双方に対応）
        try:
            # Colab環境の場合（google.colab.filesがインポートされている）
            if 'colab_files' in globals():
                print(f"\n--- ⬇️ ダウンロード実行中 ---")
                print(f"ファイル名: {csv_path.name}")
                colab_files.download(str(csv_path))
                print(f"✅ ダウンロード完了。お使いのPCのダウンロードフォルダを確認してください。")
            else:
                 raise ImportError # Colab機能が見つからない場合は次のexceptへ

        except Exception:
            # Colab環境外（ローカル環境など）の場合
            from IPython.display import FileLink, display
            print(f"\n--- 📄 ローカルファイルリンク ---")
            display(FileLink(str(csv_path.resolve())))
            print("👆 上記のリンクをクリックしてファイルをダウンロードしてください。")
else:
    print("❌ 採点に必要な 'samples' または 'generated_results' 変数が見つかりません。Cell 5/6を実行してください。")

📢 評価を開始します。10個のサンプルを採点します...
✅ LLM-as-a-judge: Scored 10 samples (0-10)


,id,general_score,specific_score,total_score
0,public_test_1,5.0,5.0,10.0
1,public_test_10,5.0,5.0,10.0
2,public_test_2,5.0,3.0,8.0
3,public_test_3,5.0,3.0,8.0
4,public_test_4,5.0,4.0,9.0
5,public_test_5,4.0,0.0,4.0
6,public_test_6,5.0,3.0,8.0
7,public_test_7,5.0,4.0,9.0
8,public_test_8,5.0,3.0,8.0
9,public_test_9,5.0,5.0,10.0


📄 Saved: outputs/runs/eval_llm_20251109_062620.csv

--- ⬇️ ダウンロード実行中 ---
ファイル名: eval_llm_20251109_062620.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ ダウンロード完了。お使いのPCのダウンロードフォルダを確認してください。
